<a href="https://colab.research.google.com/github/mmartigani/RoadMapIA/blob/main/Reiforcement_Learning_human_feedback_LLAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Reinforcement Learning from Human Feedback con PPO sobre TinyLLAMA

In [ ]:
Enunciado del caso práctico
En este caso práctico, se propone al alumno la realización de Reinforcement Learning from Human Feedback
para evitar la generación de contenido tóxico sobre una versión reducida de LLAMA denominada TinyLLAMA
Por oto lado, como algoritmo de recompensa (Reward model) se propone el uso de una
versión de RoBERTa con fine-tuning para la detección de comportamiento tóxico/hate:
https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target

In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl xformers trl evaluate sentencepiece

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Definimos los paramétros para bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
# Nombre del modelo
model_name = "PY007/TinyLlama-1.1B-Chat-v0.3"

# Leemos el modelo pre-entrenado el modelo LLAMA2-7b-chat
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
    low_cpu_mem_usage=True # Reduccion del consumo de cpu y memoria al leer el modelo
)

CHAT_EOS_TOKEN_ID = 32002

In [ ]:
from transformers import AutoTokenizer
# Leemos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
from transformers import pipeline
# Creamos un pipeline para la tokenización y generación del texto
tinyllama_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=True,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.1,
    max_new_tokens=200,
    eos_token_id=CHAT_EOS_TOKEN_ID,
)

In [ ]:
prompt = "Actúa como si fueses el mayor experto en historia del mundo. Describe \
en pocas palabras lo que ocurrió en la segunda guerra mundial."

In [ ]:
prompt_template = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
print(prompt_template)

In [ ]:
# Invocamos el pipeline para realizar generación de texto
output = tinyllama_pipe(prompt_template)
print(output[0]['generated_text'])

In [ ]:
Para este caso práctico vamos a utilizar un conjunto de datos denominado Dialogsum:
DialogSum es un conjunto de datos de resumen de diálogos a gran escala, compuesto
por 13.460 diálogos divididos en entrenamiento, prueba y validación.

In [ ]:
{'id': 'train_0', 'summary': "Mr. Smith's getting a check-up, and Doctor Hawkins advises him to have one every year. Hawkins'll give some information about their classes and medications to help Mr. Smith quit smoking.", 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.", 'topic': "get a check-up}

In [ ]:
from datasets import load_dataset
ds = load_dataset("knkarthick/dialogsum")

In [ ]:
ds

In [ ]:
# Reducimos el conjunto de datos
NUM_EJ_TRAIN = 1000
NUM_EJ_VAL = 100
NUM_EJ_TEST = 100

# Subconjunto de entrenamiento
ds['train'] = ds['train'].select(range(NUM_EJ_TRAIN))

# Subconjunto de validación
ds['validation'] = ds['validation'].select(range(NUM_EJ_VAL))

# Subconjunto de pruebas
ds['test'] = ds['test'].select(range(NUM_EJ_TEST))

In [ ]:
print(ds['train']['dialogue'][2])

In [ ]:
def prep_dataset(dataset, tokenizer, input_min_text_length, input_max_text_length):

    # Filtramos los dialogos que se encuentran entre el tamaño minimo y maximo
    dataset["train"] = dataset["train"].filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)
    dataset["validation"] = dataset["validation"].filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)
    dataset["test"] = dataset["test"].filter(lambda x: len(x["dialogue"]) > input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)

    def tokenize(sample):
        # Plantilla de entrenamiento para cada ejemplo
        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt)
        # Esto debe llamarse "query", es un requisito de la biblioteca PPO
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    # Tokenizamos cada dialogo
    dataset = dataset.map(tokenize, batched=False)

    # Convertimos el conjunto de datos a un formato adecuado
    dataset.set_format(type="torch")

    return dataset

In [ ]:
ds = prep_dataset(ds, tokenizer, input_min_text_length=200, input_max_text_length=1024)

In [ ]:
print(ds["train"]["query"][0])

In [ ]:
La siguiente función es interesante para comparar el número de parámetros
entrenables que tiene el modelo antes y después de apalicar LoRA

In [ ]:
def print_trainable_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [ ]:
print(print_trainable_parameters(model))

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

# Definición de la configuración de LoRA
lora_config = LoraConfig(
                 r = 16, # Dimensión de las matrices
                 lora_alpha = 16, # LoRA scaling factor
                 lora_dropout = 0.05, # Regularización
                 bias="none",
                 task_type="CAUSAL_LM" # Tipo de tarea/modelo al que aplicarlo
)

In [ ]:
# Aplicamos la configuración al modelo
model_peft = get_peft_model(model, lora_config)
# Mostramos el número de parámetros que se van a entrenar
model_peft.print_trainable_parameters()

In [ ]:
Durante el proceso de PPO, sólo se actualizarán algunos parámetros. En concreto, l
os parámetros entrenables con LoRA junto con algunos parámetros adicionales.
Puedes encontrar más información sobre esta clase de modelos en su documentación.
El número de parámetros entrenables puede calcularse como (𝑛+1)∗𝑚 donde 𝑛 es el
número de unidades de entrada (aquí 𝑛=2048) y 𝑚 es el número de unidades de salida
 (aquí 𝑚=1). El término +1 en la ecuación tiene en cuenta el término bias.
E nuestro caso, el número de parámetros entrenables debe ser: 2,252,800 + 2.049 = 2.254.849 parámetros

In [ ]:
from trl import AutoModelForCausalLMWithValueHead
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_peft,
                                                              torch_dtype=torch.bfloat16,
                                                              is_trainable=True,
                                                              device_map={"": 0},
)

print(f'Parametros entrenables PPO Model:\n{print_trainable_parameters(ppo_model)}\n')
print(ppo_model.v_head)

In [ ]:
from trl import create_reference_model
ref_model = create_reference_model(ppo_model)
print(f'Parámetros entrenables modelo de referencia:\n{print_trainable_parameters(ref_model)}\n')

In [ ]:
Lo siguiente que debemos hacer es selccionar el modelo de reocmpensas (Reward model).
Para este caso práctico vamos a hacer uso de una versión de RoBERTa con fine-tuning que
ha creado Meta (Facebook) para la detección de comportamiento tóxico/hate:
https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target
El modelo predecirá las probabilidades de que un texto pertenezca a una de las dos clases: (no_hate, hate)

In [ ]:
from transformers import AutoModelForSequenceClassification

reward_model_name = "facebook/roberta-hate-speech-dynabench-r4-target"

# Cargamos el modelo
reward_model = AutoModelForSequenceClassification.from_pretrained(
    reward_model_name, device_map="auto")

# Cargamos el tokenizador
reward_tokenizer = AutoTokenizer.from_pretrained(
    reward_model_name, device_map="auto")

# Etiquetas del modelo
print(f"\nEtiquetas del modelo: {reward_model.config.id2label}")

In [ ]:
def reward_evaluation(text):

  toxicity_input_ids = reward_tokenizer(text, return_tensors="pt").input_ids

  logits = reward_model(input_ids=toxicity_input_ids.to('cuda')).logits
  print(f'logits [not hate, hate]: {logits.tolist()[0]}')
  # Mostramos las probabilidades para cada categoria: [not hate, hate]
  probabilities = logits.softmax(dim=-1).tolist()[0]
  print(f'probabilities [not hate, hate]: {probabilities}')
  # Mostramos la recompensa
  not_hate_index = 0
  nothate_reward = (logits[:, not_hate_index]).tolist()
  print(f'reward (high): {nothate_reward}')

In [ ]:
# #Persona 1# le dice a Juan que no ha visto la pelicula.
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."
reward_evaluation(non_toxic_text)

In [ ]:
# #Persona 1# le dice a Tommy que la película era terrible, tonta y estúpida.
toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."
reward_evaluation(toxic_text)

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [ ]:
test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]

print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

In [ ]:
from trl import PPOConfig, PPOTrainer

learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=2
batch_size=2

config = PPOConfig(
    # model_name=model_peft,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=ds["train"],
                         data_collator=collator)

In [ ]:
Reinforcement Learning (Fine-tuning)
En este punto vamos a entrar en un bucle en el que se irán actualizando los
valores de los parámetros del modelo utilizando PPO.
El bucle consiste en los siguientes pasos principales:
Obtener los completions de LLM que se esta ajustando (modelo PEFT).
Obtener los sentimientos para las respuestas del modelo utilizando RoBERTa
Optimizar el valor de los parámetros del LLM con PPO utilizando el trío (consulta, respuesta, recompensa).
La operación se está ejecutando correctamente si ves aparecer las siguientes métricas:
objective/kl: Este valor se refiere a la divergencia de Kullback-Leibler (KL)
entre las distribuciones de probabilidad del modelo re-entrenado y el modelo de referencia.
Una divergencia KL baja sugiere que las actualizaciones de los parámetros
no están cambiando drásticamente la política, lo cual es generalmente bueno
para la estabilidad del entrenamiento.
ppo/returns/mean: Este valor representa la recompensa promedio que el
agente está obteniendo. En el aprendizaje por refuerzo, el objetivo es
generalmente maximizar la recompensa total, por lo que queremos ver este
número aumentar a lo largo del tiempo.
ppo/policy/advantages_mean: Este valor se refiere a la función de ventaja,
 que mide cuánto mejor (o peor) es tomar una acción específica en un
 estado específico, en comparación con la acción promedio en ese estado.
 Un valor de ventaja positivo sugiere que la acción es mejor que el promedio,
 y un valor negativo sugiere que es peor. Al maximizar la función de ventaja
 promedio, el algoritmo busca mejorar la política para obtener mejores recompensas.

In [ ]:
sentiment_pipe = pipeline("sentiment-analysis",
                          tokenizer=reward_tokenizer,
                          model=reward_model_name,
                          device=0) # GPU

# Argumentos proporcionados para la produción de la recompensa
reward_kwargs = {
    "top_k": None, # Return all scores.
    "function_to_apply": "none", # Set to "none" to retrieve raw logits.
    "batch_size": 2,
    "padding":'max_length',
    "truncation": True,
}

In [ ]:
print(sentiment_pipe(non_toxic_text, **reward_kwargs))

In [ ]:
from trl.core import LengthSampler
from tqdm import tqdm
import torch

output_min_length = 100
output_max_length = 300
output_length_sampler = LengthSampler(output_min_length, output_max_length)

# Argumentos proporcionados para la generación
generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
}

# Número de iteraciones durante el prceso de RL
max_ppo_steps = 15

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    # Terminamos el bucle cuando alcanzamos el máximo de iteraciones
    if step >= max_ppo_steps:
        break

    print(f"\nIteración {step} del proceso de Reinforcement Learning...")
    # Leemos los prompts de entrada para realizar la generación
    prompt_tensors = batch["input_ids"]

    # Generamos las completions del LLM (TinyLLAMA)
    summary_tensors = []
    for prompt_tensor in prompt_tensors:
        print("Procesando prompt...")
        max_new_tokens = output_length_sampler()
        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # Destokenizamos los completions. Este campo debe llamarse "response"
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in summary_tensors]
    # Mostramos por pantalla las completions
    print(f"Completions: {batch['response']}\n")
    # Calculamos la recompensa para los completions generados
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)
    # Calculamos la recompensa a partir del valor "not_hate"
    not_hate_index = 0
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]
    # Ejecutamos un paso de optimización de los parámetros de TinyLLAMA con PPO
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'\nobjective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-'.join('' for x in range(100)))

In [ ]:
# Guardamos el modelo en disco
ppo_model.save_pretrained("/content/drive/MyDrive/TinyLLAMA-ppo")

In [ ]:
# Ejemplo del conjunto de pruebas
print(ds["test"]["dialogue"][10])

In [ ]:
# Nos aseguramos de que el modelo esta en la GPU
ppo_model = ppo_model.to('cuda')
# Nos aseguramos de que el tensor de entrada esta en el formato correcto
input_ids = torch.as_tensor(ds['test']['input_ids'][10], dtype=torch.long).unsqueeze(dim=0).to('cuda')
# Argumentos proporcionados para la generación
generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "max_new_tokens": 150,
    "input_ids": input_ids
}

# Generamos la predicción
summary = ppo_model.generate(**generation_kwargs)
# Decodificamos la predicción
print(tokenizer.decode(summary.squeeze()))

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM

model_name = "PY007/TinyLlama-1.1B-Chat-v0.3"
adapters_name = "/content/drive/MyDrive/TinyLLAMA-ppo"

print(f"Cargando el modelo: '{model_name}' en memoria...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)

model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()
print(f"El modelo: '{model_name}' ha sido cargado correctamente")

In [ ]:
from transformers import AutoTokenizer
# Leemos el tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
from transformers import pipeline

CHAT_EOS_TOKEN_ID = 32002

# Creamos un pipeline para la tokenización y generación del texto
tinyllama_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    do_sample=True,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.1,
    max_new_tokens=200,
    eos_token_id=CHAT_EOS_TOKEN_ID,
)

In [ ]:
prompt = """#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath."""

In [ ]:
prompt_template = f"""
Summarize the following conversation.
{prompt}
Summary:
"""
print(prompt_template)

In [ ]:
# Invocamos el pipeline para realizar generación de texto
output = tinyllama_pipe(prompt_template)
print(output[0]['generated_text'])